# 要約 
このJupyterノートブックは、Kaggleのコンペティション「LLM 20 Questions」に参加するためのエージェントを作成するプロセスを示しています。具体的には、質問者と回答者の役割を果たすエージェントを実装し、言語モデル（LLM）を使用して質問-回答ゲームを進行することを目的としています。

### 問題の概要
このノートブックは、言語モデルを活用して「20の質問」ゲームを自動化するエージェントを構築するためのフレームワークを提供しています。エージェントは、与えられたゲームの状況に基づき、質問を生成し、またその質問に対する「はい」または「いいえ」での応答を提供する必要があります。最終的には、ターゲット単語を最速で当てることが求められます。

### 使用される手法とライブラリ
- **Gemma**: このノートブックではGoogleの`gemma_pytorch`ライブラリを使用しています。このライブラリは、因果言語モデルの訓練と推論に特化しています。
- **ImmutableDict**と**SentencePiece**: 質問と回答を効率的に処理するために`immutabledict`と`sentencepiece`パッケージをインストールしています。`immutabledict`は変更不可の辞書を提供し、`sentencepiece`はトークナイザとして使用されます。
- **PyTorch**: 言語モデルの実行にはPyTorchが利用されており、GPU上での処理が行われています。

### 主なセクション
1. **環境のセットアップ**: 必要なパッケージをインストールし、`gemma_pytorch`ライブラリを含むディレクトリを作成します。
   
2. **エージェント定義**:
   - `GemmaAgent`クラスは、言語モデルとやり取りし、プロンプトを生成して応答を得るための基本機能を持ちます。
   - `GemmaQuestionerAgent`と`GemmaAnswererAgent`クラスは、それぞれ質問者と回答者のエージェントを実装しています。これらのクラスは各ターンの処理、プロンプトのフォーマッティング、モデルからの応答解析を行います。

3. **エージェントの動作**: 
   - ゲームの状態を表すデータをもとに、エージェントのメソッドを呼び出し、適切な応答を生成します。
   - 各エージェントは、ターンタイプ（質問、推測、回答）に応じて異なるロジックを実行します。

4. **ファイルの圧縮と提出準備**: 最後に、生成されたファイルと必要なモデルファイルを一つのtarボールに圧縮し、Kaggleへの提出を容易にします。

このノートブックは、言語モデルを使用した「20の質問」ゲームの実行に必要なエージェントの構築方法を詳細に説明しており、コメントが豊富に含まれているため、実際の実装の概念を理解しやすくなっています。

---


# 用語概説 
以下は、あなたが示したJupyterノートブックの内容から、機械学習・深層学習の初心者がつまずきそうな専門用語の簡単な解説です。特にマイナーなものや実務を経験していないと馴染みのないもの、ノートブック特有のドメイン知識に焦点を当てています。

### 専門用語解説

1. **Gemma**:
   - このノートブック内で使用されている言語モデルの名称。本コンペティションにおける20の質問ゲームに特化したモデル。

2. **Causal LM (因果言語モデル)**:
   - 文章を生成する際に、過去の単語やトークンのみを考慮して次に来る単語を予測するタイプのモデル。自然言語生成において広く使われる。

3. **量子化 (Quantization)**:
   - モデルの重みや活性化値を低ビットの整数に変換する技術。これにより、モデルのサイズを縮小し、計算を効率化することができる。`get_config_for_2b`や`get_config_for_7b`はそれぞれ異なるモデルの設定を取得するための関数。

4. **Interleave (インターリーブ)**:
   - 2つのリストの要素を交互に並べる操作。リストの長さが異なる場合は、短い方のリストが終わるまで要素を交互に並べ、残りはNoneで埋める。

5. **サンプリング (Sampling)**:
   - 言語モデルからの出力生成の際に、次に生成するトークンを決定する方法。一般的には、確率的に選択されたトークンが使用され、様々なサンプリング手法（例：温度設定、top-k、top-p）がある。

6. **Few-shot examples**:
   - モデルに特定のタスクを教えるために提供される少数の例。この場合、プレイするゲームの例が含まれている。

7. **デフォルトのテンソル型 (Default Tensor Type)**:
   - PyTorchにおいて、テンソルを作成する際に使用されるデフォルトのデータ型。特定のデータ型を指定することで、メモリの使用量や計算速度を最適化することができる。

8. **contextlib.contextmanager**:
   - Pythonの標準ライブラリに含まれるモジュールで、資源の管理を簡易化するための文脈管理を提供する。特にリソースが必要な時、使い終わった時に自動的に解放されるべき場合に役立つ。

9. **tar (Tape ARchiver)**:
   - Unix系のオペレーティングシステムで、ファイルをアーカイブするためのコマンド。複数のファイルを一つのファイルにまとめる際に使用される。

10. **pigz**:
    - gzip（GNU zip）の並列実装であり、マルチコアCPUを使ってフィアルを圧縮するためのツール。大容量のファイルを迅速に圧縮できる。

11. **pv (Pipe Viewer)**:
    - パイプのデータを監視し、その進捗や統計情報を表示するためのツール。これにより、長時間かかる処理の進行状況を確認できる。

これらの用語は、ノートブック内で特有のコンテキストや技術的な背景を持つため、初心者にとっては理解するのが難しいかもしれません。

---


[スタートノートブック](https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook)を進める中で、私は友人のChatGPTと共同でこの完全に文書化されたバージョンを作成しました。コードは100％同じで、変更はありませんが、コメントが役に立つことを期待しています。

このノートブックは、**LLM 20 Questions**のエージェント作成プロセスを示しています。このノートブックを実行すると、`submission.tar.gz`ファイルが生成されます。このファイルは右側の**コンペティションに提出**という見出しから直接提出できます。あるいは、ノートブックビューワーから*Output*タブをクリックして`submission.tar.gz`を見つけ、ダウンロードします。競技ホームページの左上にある**エージェントを提出**をクリックして、ファイルをアップロードし、提出を完了させます。



In [ ]:
%%bash
cd /kaggle/working
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

上記のセルは、必要なPythonパッケージをインストールし、エージェントで使用するために`gemma_pytorch`ライブラリを準備することで、環境をセットアップします。これにより、すべての必要な依存関係が提出ファイルにバンドルされます：

- `%%bash`はJupyterノートブックで使用されるセルマジックコマンドであり、セル全体がbashスクリプトとして実行されるべきであることを示します。これにより、`%%bash`以降のすべての行が、Pythonコードではなく、シェル内でbashコマンドとして解釈されて実行されます。

- `cd /kaggle/working`は、現在の作業ディレクトリを`/kaggle/working`に変更します。このディレクトリはKaggle環境内の作業ディレクトリであり、ファイルを保存したり、ワークスペースを設定したりできます。

- `pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece`は、`immutabledict`および`sentencepiece` Pythonパッケージをインストールします。
   - `-q`はインストールプロセスからの出力を抑制します（クワイエットモード）。
   - `-U`は、すでにインストールされているパッケージをアップグレードします。
   - `-t /kaggle/working/submission/lib`は、パッケージをインストールする対象ディレクトリを指定します。これにより、依存関係が提出パッケージに含まれるようになります。

- `git clone https://github.com/google/gemma_pytorch.git > /dev/null`は、GitHubから現在のディレクトリ（`/kaggle/working`）に[`gemma_pytorch`リポジトリ](https://github.com/google/gemma_pytorch)をクローンします。
   - `> /dev/null`は`git clone`コマンドの出力を`/dev/null`にリダイレクトし、実質的にその出力をノートブックの出力から隠します。`/dev/null`は、Unix系オペレーティングシステムにおける特別なファイルで、データを記録すると捨てられ、取り出すことができません。これは、出力を抑制し、ノートブックの出力を整理して不必要な詳細を避ける方法です。

- `mkdir /kaggle/working/submission/lib/gemma/`は、`submission/lib`ディレクトリ内に`gemma`という名前の新しいディレクトリを作成します。このディレクトリには、`gemma_pytorch`リポジトリからのファイルが格納されます。

- `mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/`は、クローンした`gemma_pytorch`リポジトリの`gemma`ディレクトリ内のすべてのファイルを、新しく作成した`submission/lib/gemma/`ディレクトリに移動します。これにより、`gemma_pytorch`リポジトリから必要なファイルが提出パッケージに含まれるようになります。

これらはインストールされるパッケージです：

**immutabledict**は、変更不可の辞書を提供するパッケージです。変更不可の辞書は、作成後に辞書が変更されないことを保証する必要があるときに便利です。これは、特に構成、定数、または並行プログラミングで作業する際に重要です。

**sentencepiece**は、gemmaのトークナイザーです。



In [ ]:
%%writefile submission/main.py
# セットアップ
import os
import sys

# **重要:** コードがノートブックとシミュレーション環境の両方で機能するように、システムパスをこのように設定してください。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from pathlib import Path

# テンソル操作とモデル設定のハンドリングに必要なモジュールをインポートします。
import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

# モデルウェイトのパスを定義します。
# スクリプトがKaggleシミュレーション環境で実行されているか確認します。
if os.path.exists(KAGGLE_AGENT_PATH):
    # Kaggleシミュレーション環境で実行されている場合、ウェイトパスを適切に設定します。
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    # ローカルまたは別の環境で実行されている場合、ローカル入力ディレクトリを使用します。
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"

# プロンプトフォーマット
import itertools
from typing import Iterable


class GemmaFormatter:
    """
    20の質問ゲームのプロンプトと応答をフォーマットするためのクラス。
    """
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        """
        オプションのシステムプロンプトと少数のサンプルを使用してGemmaFormatterを初期化します。
        
        Args:
            system_prompt (str): システムの初期プロンプト。
            few_shot_examples (Iterable): プロンプトを初期化するための例。
        """
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        """
        プロンプトの現在の状態を返します。
        
        Returns:
            str: 現在のフォーマットされた状態。
        """
        return self._state

    def user(self, prompt):
        """
        現在の状態にユーザーのプロンプトを追加します。
        
        Args:
            prompt (str): ユーザーのプロンプト。
        
        Returns:
            self: GemmaFormatterインスタンス。
        """
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        """
        現在の状態にモデルのプロンプトを追加します。
        
        Args:
            prompt (str): モデルのプロンプト。
        
        Returns:
            self: GemmaFormatterインスタンス。
        """
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        """
        新しいユーザーターンを開始します。
        
        Returns:
            self: GemmaFormatterインスタンス。
        """
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        """
        新しいモデルターンを開始します。
        
        Returns:
            self: GemmaFormatterインスタンス。
        """
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        """
        現在のターンを終了します。
        
        Returns:
            self: GemmaFormatterインスタンス。
        """
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        """
        フォーマッタを初期状態にリセットします。
        提供された場合はシステムプロンプトと少数のサンプルも含みます。
        
        Returns:
            self: GemmaFormatterインスタンス。
        """
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        """
        フォーマッタに一連のターンを適用します。
        
        Args:
            turns (Iterable): 適用するターンのシーケンス。
            start_agent (str): 'user'または'model'のどちらが最初に開始されるかを指定します。
        
        Returns:
            self: GemmaFormatterインスタンス。
        """
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self


# エージェントの定義
import re


@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """
    PyTorchにおけるデフォルトのテンソル型を設定するためのコンテキストマネージャ。

    このコンテキストマネージャは、一時的に指定されたdtypeにデフォルトのテンソル型を設定し、
    コンテキストを抜けると元のデフォルトのテンソル型に戻します。

    Args:
        dtype (torch.dtype): 設定するデフォルトのテンソル型。

    Yields:
        None
    """
    # 指定されたdtypeにデフォルトのtorch dtypeを設定します。
    torch.set_default_dtype(dtype)
    yield
    # デフォルトのテンソル型をfloatに戻します
    torch.set_default_dtype(torch.float)


class GemmaAgent:
    """
    Gemma言語モデルとインタラクションするエージェントの基本クラス。
    このクラスは、モデルの初期化、プロンプトのフォーマット、応答の生成を処理します。
    """
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        """
        指定された設定でGemmaAgentを初期化します。

        Args:
            variant (str): 使用するモデルバリアント（例：'7b-it-quant'）。
            device (str): モデルを実行するデバイス（例：'cuda:0'はGPU）。
            system_prompt (str): システムの初期プロンプト。
            few_shot_examples (Iterable): プロンプトを初期化するための例。
        """
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        print("モデルを初期化しています")
        # バリアントに基づいてモデルの設定を取得
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant

        # デフォルトのテンソル型を設定し、モデルを初期化
        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt')
            model.load_weights(ckpt_path)
            self.model = model.to(self._device).eval()

    def __call__(self, obs, *args):
        """
        新しい観察を扱い、セッションを開始し、プロンプトを生成し、応答を解析します。

        Args:
            obs (dict): ゲームステート情報を含む観察辞書。
            *args: 追加の引数。

        Returns:
            str: モデルから生成された応答。
        """
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(f"{response=}")
        return response

    def _start_session(self, obs: dict):
        """
        エージェントの新しいセッションを開始します。

        Args:
            obs (dict): ゲームステート情報を含む観察辞書。

        Raises:
            NotImplementedError: このメソッドはサブクラスによって実装される必要があります。
        """
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        """
        提供されたプロンプトを使用して言語モデルから応答を生成します。

        Args:
            prompt (str): 言語モデルに対する入力プロンプト。
            max_new_tokens (int): 生成する最大新トークン数。
            **sampler_kwargs: サンプリングプロセスに対する追加のキーワード引数。

        Returns:
            str: モデルから生成された応答。
        """
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,
                'top_p': 0.1,
                'top_k': 1,
        }
        response = self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )
        return response

    def _parse_keyword(self, response: str):
        """
        モデルの応答からキーワードを抽出します。

        Args:
            response (str): モデルの応答。

        Returns:
            str: 抽出されたキーワード、見つからない場合は空の文字列。
        """
        # ダブルアスタリスク（**）で囲まれた部分文字列を見つけます。
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower()
        return keyword

    def _parse_response(self, response: str, obs: dict):
        """
        観察に基づいてモデルの応答を解析します。

        Args:
            response (str): モデルの応答。
            obs (dict): ゲームステート情報を含む観察辞書。

        Raises:
            NotImplementedError: このメソッドはサブクラスによって実装される必要があります。
        """
        raise NotImplementedError


def interleave_unequal(x, y):
    """
    2つのリストxとyをインターリーブし、欠損値のためにNoneで埋めます。
    
    この関数は、2つのリストの要素をインターリーブします。リストの長さが異なる場合は、
    欠損値のためにNoneを使用し、最終結果からこれらのNone値を除外します。

    Args:
        x (list): インターリーブする最初のリスト。
        y (list): インターリーブする2番目のリスト。

    Returns:
        list: None値を除外してインターリーブされたxとyの要素を持つリスト。
    
    例:
        >>> interleave_unequal([1, 2, 3], ['a', 'b'])
        [1, 'a', 2, 'b', 3]
    """
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]


class GemmaQuestionerAgent(GemmaAgent):
    """
    20の質問ゲームの質問者の役割を果たすエージェント。
    
    このエージェントは、ゲームのステートに基づいて質問を生成し、言語モデルからの応答を解析して
    質問プロセスを進めます。
    """
    def __init__(self, *args, **kwargs):
        """
        提供された引数でGemmaQuestionerAgentを初期化します。
        
        Args:
            *args: 基本クラスの初期化子に渡す位置引数。
            **kwargs: 基本クラスの初期化子に渡すキーワード引数。
        """
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        """
        質問者エージェントの新しいセッションを開始し、フォーマッタをリセットし、
        初期プロンプトと以前のターンを適用します。
        
        Args:
            obs (dict): ゲームステート情報を含む観察辞書。
                - obs.questions (list): 以前にした質問のリスト。
                - obs.answers (list): 受け取った対応する回答のリスト。
                - obs.turnType (str): 現在のターンのタイプ（'ask'または'guess'）。
        """
        self.formatter.reset()
        self.formatter.user("20の質問をしましょう。あなたは質問者の役割を果たします。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("はいまたはいいえで答えられる質問をしてください。")
        elif obs.turnType == 'guess':
            self.formatter.user("今、キーワードを推測してください。推測はダブルアスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        """
        ゲームのターンのタイプに基づいてモデルの応答を解析します。
        
        Args:
            response (str): 言語モデルによって生成された応答。
            obs (dict): ゲームステート情報を含む観察辞書。
                - obs.turnType (str): 現在のターンのタイプ（'ask'または'guess'）。
        
        Returns:
            str: ターンタイプに基づいて解析された質問または推測。
        
        Raises:
            ValueError: 観察内のターンタイプが不明である場合。
        """
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None:
                question = "それは人ですか？"
            else:
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else:
            raise ValueError("不明なターンタイプ:", obs.turnType)


class GemmaAnswererAgent(GemmaAgent):
    """
    20の質問ゲームの回答者の役割を果たすエージェント。
    
    このエージェントは、ゲームのステートに基づいて「はい」または「いいえ」の回答を提供し、
    言語モデルからの応答を解析して回答プロセスを続けます。
    """
    def __init__(self, *args, **kwargs):
        """
        提供された引数でGemmaAnswererAgentを初期化します。
        
        Args:
            *args: 基本クラスの初期化子に渡す位置引数。
            **kwargs: 基本クラスの初期化子に渡すキーワード引数。
        """
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        """
        回答者エージェントの新しいセッションを開始し、フォーマッタをリセットし、
        初期プロンプトと以前のターンを適用します。
        
        Args:
            obs (dict): ゲームステート情報を含む観察辞書。
                - obs.questions (list): 以前にした質問のリスト。
                - obs.answers (list): 与えられた対応する回答のリスト。
                - obs.keyword (str): 質問者が推測しているキーワード。
                - obs.category (str): キーワードのカテゴリ。
        """
        self.formatter.reset()
        self.formatter.user(f"20の質問をしましょう。あなたは回答者の役割を果たします。キーワードは{obs.keyword}で、カテゴリは{obs.category}です。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"その質問はキーワード{obs.keyword}についてで、カテゴリは{obs.category}です。「はい」または「いいえ」で答えて、回答をダブルアスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        """
        モデルの応答を解析して「はい」または「いいえ」の回答を抽出します。
        
        Args:
            response (str): 言語モデルによって生成された応答。
            obs (dict): ゲームステート情報を含む観察辞書。
        
        Returns:
            str: 'yes'（回答に「yes」が含まれている場合）、それ以外は「no」。
        """
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'


# エージェントの作成
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを考え、質問者の質問に「はい」または「いいえ」で答えます。キーワードは特定の人、場所、または物です。"

few_shot_examples = [
    "20の質問をしましょう。あなたは質問者の役割を果たします。最初の質問をしてください。",
    "それは人ですか？", "**いいえ**",
    "それは場所ですか？", "**はい**",
    "それは国ですか？", "**はい** ではキーワードを推測してください。",
    "**フランス**", "正解です！",
]

# **重要:** 必要なエージェントを一つだけロードできるようにエージェントをグローバルとして定義します。
# 両方のエージェントをロードすると、メモリ不足になる可能性があります。
agent = None


def get_agent(name: str):
    """
    提供された名前に基づいて、適切なエージェント（質問者または回答者）を初期化して返します。
    
    この関数はグローバル変数を使用して、エージェントのインスタンスが一つだけ作成されることを保証します。
    エージェントがまだ初期化されていない場合、提供された名前に基づいてGemmaQuestionerAgentまたはGemmaAnswererAgentの新しいインスタンスを作成します。
    
    Args:
        name (str): 初期化するエージェントの名前（'questioner'または'answerer'）。

    Returns:
        GemmaAgent: GemmaQuestionerAgentまたはGemmaAnswererAgentのインスタンス。
    
    Raises:
        AssertionError: エージェント名が認識されない場合、またはエージェントの初期化に失敗した場合。
    """
    global agent
    
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    assert agent is not None, "エージェントが初期化されていません。"

    return agent


def agent_fn(obs, cfg):
    """
   ターンのタイプを判定し、適切なエージェントを呼び出して応答を生成します。
    
    この関数は観察内のターンタイプを調べ、その対応するエージェント（質問者または回答者）を使用して応答を生成します。
    応答がNoneまたは空である場合、"yes"を返します。
    
    Args:
        obs (dict): ゲームステート情報を含む観察辞書。
            - obs.turnType (str): 現在のターンのタイプ（'ask'、'guess'、または'answer'）。
        cfg (dict): エージェントの設定（現在の実装では使用されていません）。

    Returns:
        str: エージェントによって生成された応答、または応答がNoneまたは空である場合は"yes"。
    """
    if obs.turnType == "ask":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "guess":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)
    if response is None or len(response) <= 1:
        return "yes"
    else:
        return response

In [ ]:
!apt install pigz pv > /dev/null

上記のセルは、高速圧縮のための`pigz`と、進捗監視のための`pv`をインストールします：

- **apt install pigz pv**: `pigz`（gzipの並列実装）と`pv`（パイプビューア、データのパイプラインを通る進捗を監視可能）をインストールします。
- **> /dev/null**: コマンドの出力を`/dev/null`にリダイレクトし、出力を抑制してノートブックをクリーンに保ちます。



In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2

上記のセルは、提出ディレクトリと必要なモデルファイルを圧縮tarボール（`submission.tar.gz`）内にパッケージ化します。このtarボールには以下が含まれます：
   - `/kaggle/working/submission`内のすべてのファイル。
   - `/kaggle/input`からの`gemma/pytorch/7b-it-quant/2`ディレクトリ。
   
詳細な内訳：
- **--use-compress-program='pigz --fast --recursive | pv'**: 圧縮に`pigz`を使用するように指定し、複数のCPUコアを利用してプロセスを高速化します。`--fast`オプションは迅速な圧縮を保証し、`--recursive`はディレクトリを再帰的に処理します。出力は`pv`を通してパイプされ、進捗を監視します。
- **-cf submission.tar.gz**: `submission.tar.gz`という名前のファイルを作成します。
- **-C /kaggle/working/submission**: アーカイブにファイルを追加する前に、ディレクトリを`/kaggle/working/submission`に変更します。
- **.**: 現在のディレクトリ（オプションの`-C`によって`/kaggle/working/submission`になっています）からすべてのファイルをアーカイブに追加します。
- **-C /kaggle/input/**: より多くのファイルを追加する前に、`/kaggle/input/`ディレクトリに変更します。
- **gemma/pytorch/7b-it-quant/2**: `/kaggle/input/`から`gemma/pytorch/7b-it-quant/2`ディレクトリとその内容をアーカイブに追加します。